<div class="alert alert-block alert-danger">
TODO change to the  code from lectures and change  progression:
    <ol> 
        <li> Activity 1: the data </li>
         <ul> 
             <li>Iris data using the actua llabels then the visualisation from last week</li>
            <li> student marks data using visualiations from last week </li>    
            <li> MCQ's to confirm that one class is seperable, the others are not </li>
        </ul>
    <li>  Activity 2: Take my 1-NN code from the lectures, and the k-NN pseudocode and implement kNN for iris and studentMarks</li>
        <ul>
          <li> Sanity check: create a new classifier using the sklearn class and verify their results are in the same ball park (not statistically significantly different?)  </li>
          <li> provide the code that plots decision surfaces, look at effect of changing K </li>
        </ul>
        
    <li> Actvity 3: repeat for decision trees using code from lectures.</li>
        <ul> 
            <li>Task is to examine effect of changing max depth and number per split on (i) trees and. (ii) test accuracy</li>
            <li> needs MCQ to test findings  and understnading - max-depth limits every subtree,  min-spolit limiits unpopulated subtrees</li>
        </ul>
    </ol></div>

<div class="alert alert-block alert-danger">
TODO change to the  code from lectures and change  progression:
    <ol> 
    <li>  They put the Iris data into the visualisations they made last week but this time using the class labels (provbide rough code ion case they didn;t do last week but encourage thme to use their own)</li>
        <li> MCQ's to confirm that one class is seperable, the others are not </li>
    <li>  Take my 1-NN code from the lectures, and the k-NN pseudocode and implement kNN for iris</li>
    <li> Sanity check: create a nre wclassifier using the sklearn class and verify their results are in the same ball park (not statistically significantly different?)  </li>
<li> Wrap-up discussion about comparing algorithms and when it is safe to say methodX is better than method Y. Realte back to disc ussinos in new second week workbook, topical real world - relate to comparison of vaccines, or different treatments for covid
    </ol></div>

# Workbook 6: Supervised Machine Learning

## Description and aims

This tutorial is designed to give you your first experience of machine learning in practice by implementing a simple nearest-neighbour classifier.

The learning outcomes are:
- experience of implementing the K Nearest Neighbours classification algorithm
- experience of using the sklearn DecisionTree classification algorithm
-  experience of working through different preprocessing steps to try and improve the performance of your classifier

## Activity 1: Getting to know your data: 

We will start by importing and visualising the two datasets used as examples in the lecture: students marks,  and Iris
### You should already have uploaded the data and figures from the lecture materials folder - if not, do so now

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def show_scatterplot_matrix(X,y,featureNames,title=None):
    f = X.shape[1]
    if(len(y) != X.shape[0]):
        print("Error,   the y array  must have the same length as there are rows in X")
        return
    fig, ax = plt.subplots(f,f,figsize=(12,12))
    plt.set_cmap('jet')
    for feature1 in range(f):
        ax[feature1,0].set_ylabel( featureNames[feature1])
        ax[0,feature1].set_xlabel( featureNames[feature1])
        ax[0,feature1].xaxis.set_label_position('top') 
        for feature2 in range(f):
            xdata = X[:,feature1]
            ydata = X[:,feature2]
            ax[feature1, feature2].scatter(xdata,ydata,c=y)
    if title != None:
        fig.suptitle(title,fontsize=16,y=0.925)
            

### The Student marks dataset

In [ ]:

grades= np.genfromtxt("../lectures/data/assessment-grades-2features.csv", delimiter= ',',skip_header=1)

featureNames=("exam", "CW_mean")
nStudents = grades.shape[0]

outcomes= ("Pass","Resit Exam", "Resit Coursework","Resit Both")
simpleoutcomes= ("pass","resit")

# make target labels
result = np.empty(nStudents, dtype=np.int8)

for row in range (nStudents):
    exam = grades[row][0]
    cw   = grades[row][1]
    if (exam>=35 and cw>=35 and (exam +cw >=80) ):
        result[row] = 0 # PASS 

    elif ( cw>=40 and exam < 40):
        result[row] = 1 #resit just exam 
    elif ( cw<40 and exam>=40):
        result[row]= 2 # resit just coursework
    else:
        result[row] = 3  # resit both
        
simpleResult = np.where(result<1,0,1)

In [ ]:
# easiest to split the data into 4/2 subgroups ot plot the outomes /simplified outcomes

passStudents = np.empty((0,2))
resitCWStudents = np.empty((0,2))
resitExamStudents = np.empty((0,2))
resitBothStudents = np.empty((0,2))

for student in range (nStudents):
    if (result[student]==0):
        passStudents = np.vstack( (passStudents,grades[student]) )
    elif (result[student]==1):
        resitExamStudents = np.vstack( (resitExamStudents,grades[student]) )
    elif (result[student]==2):
        resitCWStudents = np.vstack( (resitCWStudents,grades[student]) )
    else:
        resitBothStudents = np.vstack( (resitBothStudents,grades[student]) )
simpleResitStudents = np.vstack( (resitExamStudents,resitCWStudents,resitBothStudents))

print(passStudents.shape)
print(resitExamStudents.shape)
print(resitCWStudents.shape)
print(resitBothStudents.shape)
print(simpleResitStudents.shape)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(14,5))
plt.xlabel("Exam")
plt.ylabel("Coursework")
ax[0].set_title("Outcomes")
ax[1].set_title("Simplified Outcomes")

ax[0].scatter(passStudents[:,0],passStudents[:,1],label = "Pass" )
ax[0].scatter(resitExamStudents[:,0],resitExamStudents[:,1],label = "Resit Exam" )
ax[0].scatter(resitCWStudents[:,0],resitCWStudents[:,1],label = "Resit CW" )
ax[0].scatter(resitBothStudents[:,0],resitBothStudents[:,1],label = "Resit Both" )
ax[1].scatter(passStudents[:,0],passStudents[:,1],label = "Resit" )
ax[1].scatter(simpleResitStudents[:,0],simpleResitStudents[:,1],label = "Pass" )

ax[0].legend(loc='lower right')
ax[1].legend(loc='lower right') 

### Example 2:  Iris flowers <img src="../lectures/figures/ML/Iris-image.png" style="float:right">
- classic Machine Learning Data set
- 4 measurements: sepal and petal width and length
- 50 examples  from each 3 sub-species for iris flowers
- three class problem:
 - so for some types of algorithm have to decide whether to make  
   a 3-way classifier or nested 1-vs-rest classifers
- most ML classifiers can get over 90%



In [ ]:
import sklearn.datasets
irisX,irisy = sklearn.datasets.load_iris(return_X_y=True)
columnLabels= ("sepal_length", "sepal_width", "petal_length", "petal_width")
title="Scatterplots of 2D slices through the 4D Iris data"
show_scatterplot_matrix(irisX,irisy,columnLabels,title)

## Activity 2: Implementing K-Nearest Neighbours
Below is the pseudocodefor the K-nearest Neighbours algorithm.
Make sure you understand this,   then read the cell below which is my implentation for K=1

### Pseudocode for KNearest Neighbours
**init()**  :  
Specify a distance metric d(i,j) for any two items *i* and *j*     
  e.g. Euclidean (continuous variables) or Hamming (categorical)

**fit(trainingData)** :  
Just store a local copy of the training data as two arrays:  
X_train of shape (numTrainingItems , numFeatures),  
y_train of shape( numTrainingItems)
  
**predict(newItems)** :  
*Step 1:   Make 2D array distances of shape (num_newItems , numTrainingItems)*   
FOREACH newItem i  
...FOREACH trainingitem j  
.....SET distances [i] [j] = d (i,j) 

*Step 2: Get labels of the k nearest neighbours*  
FOREACH newItem i  
...Find the *k* columns for row i with the smallest values  
...Get the corresponding *k* labels from y_train  

*Step 3: Store majority vote in a  1D array y_pred of size (numToPredict)*   
FOREACH newItem i  
...FOREACH label m  
......Count votes amongst the k Nearest neightbour of i  
...... SET y_pred[i] = value of m with highest count
 
RETURN y_pred

In [ ]:
# Example for K = 1 
from sklearn.metrics.pairwise import euclidean_distances
class simple_1NN:

    def fit(self,X,y):
        self.numExemplars = X.shape[0]
        self.numFeatures = X.shape[1]
        self.modelX = X
        self.modelY = y
        
    def predict(self,newItems):
        numToPredict = newItems.shape[0]
        yPred = np.zeros((numToPredict,1))
        
        # measure distances - creates an array with numToPredict rows and num_trainItems columns
        dist = euclidean_distances(newItems,self.modelX)

        #make predictions: This is K=1, TO DO- in your own time extend to work with K>1
        for item in range(numToPredict):
            closest = np.argmin(dist, axis=1) 
            yPred[item] = self.modelY [ closest[item]]
        
        return yPred

### Activity 2.1 Run the code provided for K=1 with the two datasets and make sure you understand the outputs and how they are produced
- for the marks dataset this creates a plot to show a decision surface
- for the  iris data set this uses a confision matrix

**The Marks dataset - illustrating a 2D Decision surface**

In [ ]:
#create and prettify the plot
cmap="Set3"
fig,ax= plt.subplots(figsize=(8, 8))
ax.set_title("1 nearest neighbours, simplified results")
ax.set_xlabel(featureNames[0])
ax.set_ylabel(featureNames[1])

#define a grid we use to plot the decision boundaries
h = 2.0    
x_min, x_max = -10,110
y_min, y_max = -10,110
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))

# create and train the classifier
myKNN = simple_1NN()
myKNN.fit(grades,simpleResult) 

#predict and plotfor evey point on the grid
Z = myKNN.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z,cmap=cmap)

# Plot also the training points
ax.scatter(x=grades[:,0 ],y= grades[:, 1], c=simpleResult.astype(float), alpha=1.0, cmap=cmap, edgecolor="black")

**The Iris dataset - illustrating a confusion matrix**

In [ ]:
# make train/test split 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

irisX,irisy = load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(irisX, irisy, test_size=0.33,stratify=irisy)


model = simple_1NN()
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)



### Activity 2.2 (stretch): edit the code in the cells above to produce:
- a confusion matrix for the marks dataset
- a decision surface for the four-class version of the marks dataset  
  i.e. using the labels held in the array "results" instead of "simplifiedResults"
- a decision surface for the Iris Data  
  you will need to take just two features - you could use the first two, or the best two you identified last week

### Activity 2.3: Create your own implementation of K-Nearest Neighbours
Using the code above,  extend it to use the votes from K>1 neighbours.

These are the lines you will need to change:
````  
#make predictions: This is K=1, TO DO- in your own time extend to work with K>1
        for item in range(numToPredict):
            closest = np.argmin(dist, axis=1) 
            yPred[item] = self.modelY [ closest[item]]
```` 

Some hints: 
- Make sure you change the class name to something appropriate
- You should  set (and store) the value of K in the constructor method
- If I have an array of myDist of (say, for simplicity) five values,  
   and I want to resort them by size, keep track of the item ids  
   so i can find the K with the smallest values in the original array. 
   I can do it using the code below, which:
   - creates a 2d array holding each value and its index in the original array
   - then makes a new array which is the 'unSorted' 2d array, but sorted by the values in the first olumn ([0])
   - then looks in the second column (which holds the indexes) for the first K rows

In [ ]:
# This cells just contains some hints about how to prodice a sorted array
myDist = np.array([4,7,1,3,9])

myUnsortedArray = np.empty((5,2))
print(myUnsortedArray.shape)
for row in range (5):
    myUnsortedArray [row][0] = myDist[row]
    myUnsortedArray [row][1] = row
print('myUnsortedArray contents before sorting')
print(myUnsortedArray)

ind = np.argsort( myUnsortedArray[:,0] );
mySortedArray = myUnsortedArray[ind]


print('mySortedArray contents after sorting')
print(mySortedArray)

print('Last week we learned about slicing- we will now use slicing to pull out all rows but just specific columns')
print(' the values in array myDist in ascending order are: {}'.format(mySortedArray[:,0]))
print(' the positions those values were in, again by ascending order of value are : {}'.format(mySortedArray[:,1]))
print('Now remember you do not have to choose every row ... ')

In [ ]:
# your KNN class code here

### Activity 2.4: Test your impementation on the two example datasets
- use the toolbar to copy and paste the two cells from activity 2.1 below here
- then edit them so that they create and use objects of your new class, instead of the class simple_1NN

- start with K=1 - this should produce the same results as you got in activity 2.1
- then try with K = {3,5,7}
- what happens to the accuracy?
- what happens to the decision surface?


## Activity 3: Decision Trees


The ainm of this activity is for you to experiment with what happens when you change three parameters that affect how big and complex the tree is allowed to get.
- max_depth
- min_samples_split, (default value is 2)
- min_samples_leaf, (default value is 1)

Experiment with the data sets below to see if you can work out what each of these parameters does, and how it affects the tree 

- Each time you run the  cell below, it will give you a different train-test split of the Iris data.
  Does this affect what tree you get?
  
- Is there a combination of values that means you consistently get similar trees?
- What is a good way of judgiung 'similarity?




In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

# load iris dataset and split into train:test
iris = sklearn.datasets.load_iris()
irisX = iris.data
irisy = iris.target

X_train, X_test, y_train, y_test = train_test_split(irisX, irisy, test_size=0.33,stratify=irisy)

model = DecisionTreeClassifier(random_state=1234, max_depth=None,min_samples_split=2,min_samples_leaf=1)
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)



fig = plt.figure(figsize=(12,12))
_ = tree.plot_tree(model, 
                   feature_names=iris.feature_names,  
                   class_names=iris.target_names,
                   filled=True)


<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>